# Mini Project 3: Searching for Tweets

### For this mini project, I'll be using the following:
#### Elasticsearch
#### Tweepy
#### json

In [1]:
import elasticsearch
from elasticsearch import Elasticsearch                    

In [2]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import time
import json

In [3]:
c_key = 'wwwKNtSMuTHqgL0KLp9RFqySu'
secret_c_key = 'ekkrXaxm3zOfGSIoQuMckt3I3ig5HBNw8uZgZYLgswASV8xWqP'
access_token = '846115336877432832-M7UoANTmDy2uzOAHVN8WgIuOnSd7L6g'
secret_access_token = 'Qi7fahFkg9LfM3PRLurqXZkmcxNWgCHmKz39Jn4gTdFTK'

### These keys and tokens will probably be changed by the time I upload the nbviewr file.

In [4]:
import logging
logging.getLogger("elasticsearch").setLevel(logging.CRITICAL)
logging.basicConfig(level=logging.CRITICAL)

In [6]:
ES_HOST = {"host": "localhost", "port": 9200}
elast = Elasticsearch(hosts=[ES_HOST])

In [9]:
class Listener(StreamListener):
    def __init__(self, number):
        self.counter = 0
        self.number = number

    def on_data(self, data):
        dict_data = json.loads(data)
        print (dict_data["text"])
        elast.index(index="twitter",
                 doc_type="stream",
                 body={"author": dict_data["user"]["screen_name"],
                       "date": dict_data["created_at"],
                       "message": dict_data["text"]})
        self.counter +=1
        if self.counter == self.number:
            return False
        
        return True
    
    def on_error(self, status):
        print (status)
                       
if __name__ == '__main__':

    Tweet_Listening = Listener(number = 15) 
    Auth = OAuthHandler(c_key, secret_c_key)
    Auth.set_access_token(access_token, secret_access_token)
    Streaming = Stream(Auth, Tweet_Listening)
    Streaming.filter(track=['pizza', 'hamburger'])

RT @Marruecos67: Rajoute un t-max aussi https://t.co/oqFMP5BPcd
NO FAIR!!! whiting out the board. NO FAIR!!!
RT @cpeckk: i recently discovered that i can get chuck e cheese pizza delivered to my house via postmates and i really just want to know wh…
Para que no me vaya de mi casa mi mamá me compra pizza . Que trucazo
RT @Jiminpicss: friendly reminder that jimin bites his ice cream, puts milk before cereal and likes pineapple on his pizza.
YES YES YES
The target Reuben pizza bowl
RT @imajsaclaimant: This is great... Khan Pizza takeaway in Hull to open its doors on Christmas Day to give out essential items and hot foo…
RT @pizzahutuk: Hi @ManUtd, we've received a CV from Mr J Mourinho. 
He's just applied for a job as pizza chef.
Please can you send through…
@imnayyaeon Mau peje pizza sm matcha :(
Open NOW until Friday night for "OVERTIME! Customer Appreciation Days!!". Hit the SEND MESSAGE button or call/text… https://t.co/q7Lp6jvpXF
RT @pizzahutuk: Hi @ManUtd, we've received a CV from M

### The fun part is that whenever I want to execute another search, I have to uninstall and reinstall Elasticsearch. I'm sure there's some file or folder I could delete that would have the same effect, but I don't want to spend hours testing which one that would be. Regardless, I have my tweets, and I'm happy.

### This also explains why there is a huge gap between the numbers for the Elasticsearch part and the query part, I don't want to restart my Python3 code because I'd have to reinstall Elasticsearch. Anyway, for this next line, I just wanted to explore if Elastisearch created nodes with the indices I set for it earlier.

In [85]:
for index in elast.indices.get('*'):
  print (index)

.kibana_1
twitter


### Now for the search itself: to find the tweets in our Elasticsearch index that do not have a source field.

In [86]:
res = elast.search(index = "twitter", doc_type = "stream", _source = True)
print (res)

{'took': 2, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': 15, 'max_score': 1.0, 'hits': [{'_index': 'twitter', '_type': 'stream', '_id': '1VZzw2cBcHlmTvfkD-dd', '_score': 1.0, '_source': {'author': 'noregre73766334', 'date': 'Tue Dec 18 22:32:31 +0000 2018', 'message': 'RT @Marruecos67: Rajoute un t-max aussi https://t.co/oqFMP5BPcd'}}, {'_index': 'twitter', '_type': 'stream', '_id': '2VZzw2cBcHlmTvfkFedp', '_score': 1.0, '_source': {'author': 'joonbootie', 'date': 'Tue Dec 18 22:32:33 +0000 2018', 'message': 'RT @Jiminpicss: friendly reminder that jimin bites his ice cream, puts milk before cereal and likes pineapple on his pizza.'}}, {'_index': 'twitter', '_type': 'stream', '_id': '4VZzw2cBcHlmTvfkKOcM', '_score': 1.0, '_source': {'author': 'Carmdee66Carman', 'date': 'Tue Dec 18 22:32:38 +0000 2018', 'message': 'I liked a @YouTube video https://t.co/WBXdtu3F6V My first Pompeii Brick Pizza Oven - Time Lapse Video'}}, {'_inde

### So out of the tweets we gathered, all of them appear here. We can still narrow this down by adding in one of the parameters listed.

In [87]:
res2 = elast.search(index = "twitter", doc_type = "stream", body={"query": {"match": {"_id":"1VZzw2cBcHlmTvfkD-dd"}}})
print (res2)

{'took': 2, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': 1, 'max_score': 1.0, 'hits': [{'_index': 'twitter', '_type': 'stream', '_id': '1VZzw2cBcHlmTvfkD-dd', '_score': 1.0, '_source': {'author': 'noregre73766334', 'date': 'Tue Dec 18 22:32:31 +0000 2018', 'message': 'RT @Marruecos67: Rajoute un t-max aussi https://t.co/oqFMP5BPcd'}}]}}


### We've narrowed down the tweets form our Elasticsearch module to one with a specific ID through the search command here.